In [1]:
import dotenv
import requests
import os

dotenv.load_dotenv()

PHUNT_API_TOKEN = os.environ["PHUNT_API_TOKEN"]

def fetch_posts(n_posts: int):
    url = "https://api.producthunt.com/v2/api/graphql"
    headers = {'Authorization': 'Bearer {}'.format(PHUNT_API_TOKEN)}
    data = {
        "query":
            """{
              posts(first: %d) {
                edges {
                  node {
                    id
                    name
                    description
                    url
                    votesCount
                    topics {
                      edges {
                        node {
                          name
                        }
                      }
                    }
                  }
                }
              }
            }""" % n_posts
        }
    r = requests.post(url, json = data, headers=headers)
    result = r.json()
    return result

In [2]:
response = fetch_posts(10)

In [3]:
response

{'data': {'posts': {'edges': [{'node': {'id': '452266',
      'name': 'Grimo AI (Alpha)',
      'description': 'Grimo is not just another notebook. It is built for crazy learners and creators. It is where you get access to serious and valuable knowledge, without leaving the notebook. Less time on searching and organizing knowledge. More on learning.',
      'url': 'https://www.producthunt.com/posts/grimo-ai-alpha?utm_campaign=producthunt-api&utm_medium=api-v2&utm_source=Application%3A+dystopian_index+%28ID%3A+122213%29',
      'votesCount': 65,
      'topics': {'edges': [{'node': {'name': 'Productivity'}},
        {'node': {'name': 'Notes'}},
        {'node': {'name': 'Artificial Intelligence'}}]}}},
    {'node': {'id': '453029',
      'name': 'Parny',
      'description': 'Parny is the all-in-one solution for on-call management and monitoring alert services. It leverages a social media-style experience and AI-powered solutions.',
      'url': 'https://www.producthunt.com/posts/parny?u

In [4]:
product_list = response["data"]["posts"]["edges"]
product_list

[{'node': {'id': '452266',
   'name': 'Grimo AI (Alpha)',
   'description': 'Grimo is not just another notebook. It is built for crazy learners and creators. It is where you get access to serious and valuable knowledge, without leaving the notebook. Less time on searching and organizing knowledge. More on learning.',
   'url': 'https://www.producthunt.com/posts/grimo-ai-alpha?utm_campaign=producthunt-api&utm_medium=api-v2&utm_source=Application%3A+dystopian_index+%28ID%3A+122213%29',
   'votesCount': 65,
   'topics': {'edges': [{'node': {'name': 'Productivity'}},
     {'node': {'name': 'Notes'}},
     {'node': {'name': 'Artificial Intelligence'}}]}}},
 {'node': {'id': '453029',
   'name': 'Parny',
   'description': 'Parny is the all-in-one solution for on-call management and monitoring alert services. It leverages a social media-style experience and AI-powered solutions.',
   'url': 'https://www.producthunt.com/posts/parny?utm_campaign=producthunt-api&utm_medium=api-v2&utm_source=Appli

In [5]:
from pydantic import BaseModel

class PHuntProduct(BaseModel):
    id: str
    name: str
    description: str
    url: str
    n_votes: int
    prod_topics: str

In [6]:
product_dict = product_list[0]["node"]
product_dict["n_votes"] = product_dict["votesCount"]
product_dict["prod_topics"] = ", ".join([i["node"]["name"] for i in product_dict["topics"]["edges"]])

In [7]:
PHuntProduct.model_validate(product_dict)

PHuntProduct(id='452266', name='Grimo AI (Alpha)', description='Grimo is not just another notebook. It is built for crazy learners and creators. It is where you get access to serious and valuable knowledge, without leaving the notebook. Less time on searching and organizing knowledge. More on learning.', url='https://www.producthunt.com/posts/grimo-ai-alpha?utm_campaign=producthunt-api&utm_medium=api-v2&utm_source=Application%3A+dystopian_index+%28ID%3A+122213%29', n_votes=65, prod_topics='Productivity, Notes, Artificial Intelligence')

In [8]:
for product in product_list:
    product_dict = product["node"]
    product_dict["n_votes"] = product_dict["votesCount"]
    product_dict["prod_topics"] = ", ".join([i["node"]["name"] for i in product_dict["topics"]["edges"]])
    product_model = PHuntProduct.model_validate(product_dict)
    print(product_model)

id='452266' name='Grimo AI (Alpha)' description='Grimo is not just another notebook. It is built for crazy learners and creators. It is where you get access to serious and valuable knowledge, without leaving the notebook. Less time on searching and organizing knowledge. More on learning.' url='https://www.producthunt.com/posts/grimo-ai-alpha?utm_campaign=producthunt-api&utm_medium=api-v2&utm_source=Application%3A+dystopian_index+%28ID%3A+122213%29' n_votes=65 prod_topics='Productivity, Notes, Artificial Intelligence'
id='453029' name='Parny' description='Parny is the all-in-one solution for on-call management and monitoring alert services. It leverages a social media-style experience and AI-powered solutions.' url='https://www.producthunt.com/posts/parny?utm_campaign=producthunt-api&utm_medium=api-v2&utm_source=Application%3A+dystopian_index+%28ID%3A+122213%29' n_votes=63 prod_topics='Productivity, Developer Tools, Tech'
id='450036' name='Stevie AI' description='Stevie AI is an SEO Ass

### Try download script

In [9]:
from src.data_ingestion.download_product_hunt import fetch_ph_posts, parse_ph_response


response = fetch_ph_posts(10)
product_list = parse_ph_response(response)

In [10]:
product_list

[PHuntProduct(id='453029', name='Parny', description='Parny is the all-in-one solution for on-call management and monitoring alert services. It leverages a social media-style experience and AI-powered solutions.', url='https://www.producthunt.com/posts/parny?utm_campaign=producthunt-api&utm_medium=api-v2&utm_source=Application%3A+dystopian_index+%28ID%3A+122213%29', n_votes=69, prod_topics='Productivity, Developer Tools, Tech'),
 PHuntProduct(id='452266', name='Grimo AI (Alpha)', description='Grimo is not just another notebook. It is built for crazy learners and creators. It is where you get access to serious and valuable knowledge, without leaving the notebook. Less time on searching and organizing knowledge. More on learning.', url='https://www.producthunt.com/posts/grimo-ai-alpha?utm_campaign=producthunt-api&utm_medium=api-v2&utm_source=Application%3A+dystopian_index+%28ID%3A+122213%29', n_votes=68, prod_topics='Productivity, Notes, Artificial Intelligence'),
 PHuntProduct(id='45003

In [15]:
import pandas as pd

df = pd.DataFrame([i.model_dump() for i in product_list])
df

,id,name,description,url,n_votes,prod_topics
0,453029,Parny,Parny is the all-in-one solution for on-call m...,https://www.producthunt.com/posts/parny?utm_ca...,69,"Productivity, Developer Tools, Tech"
1,452266,Grimo AI (Alpha),Grimo is not just another notebook. It is buil...,https://www.producthunt.com/posts/grimo-ai-alp...,68,"Productivity, Notes, Artificial Intelligence"
2,450036,Stevie AI,Stevie AI is an SEO Assistant that helps any n...,https://www.producthunt.com/posts/stevie-ai?ut...,56,SEO
3,449283,Promomix,Don't know what to say after shooting a UGC vi...,https://www.producthunt.com/posts/promomix?utm...,47,"Social Media, Artificial Intelligence, Audio"
4,452339,SnapSign,SnapSign provides a simple and affordable solu...,https://www.producthunt.com/posts/snapsign?utm...,41,"Productivity, Photography, Video"
5,453073,KickRender,KickRender is a 3D rendering tool powered by A...,https://www.producthunt.com/posts/kickrender?u...,47,"Design Tools, 3D Modeling, Interior design"
6,452546,Sphēra: Emotion Tracker,We don't believe in labeling emotions as right...,https://www.producthunt.com/posts/sphera-emoti...,43,Health & Fitness
7,452961,Vidnoz AI 2.8,"①600+ AI avatars, 470+AI voices, 700+ video te...",https://www.producthunt.com/posts/vidnoz-ai-2-...,42,"Artificial Intelligence, Video"
8,452935,Album Cleaner: SwipeClea,SwipeClean is the ultimate solution for organi...,https://www.producthunt.com/posts/album-cleane...,41,"Apple, Video, Photo editing"
9,443760,WebCurate,An advanced directory where you can showcase y...,https://www.producthunt.com/posts/webcurate?ut...,32,"Design Tools, Developer Tools, Maker Tools"
